In [5]:
import cv2
import tkinter as tk
from threading import Thread
import re
import time
import pywhatkit as kit

In [ ]:
# Declarar las variables globales para mantener un seguimiento del estado de la cámara y el último dato leído
camera_active = False
capture = None  # Variable para almacenar la captura de la cámara
camera_thread = None
last_data = ""
celular_detectado = ""

def iniciar_camara():
    global camera_active, capture, camera_thread, celular_detectado
    if not camera_active:
        capture = cv2.VideoCapture(0)
        camera_active = True

        # Desactivar el botón "Iniciar Cámara" después de iniciar la cámara
        start_button.config(state=tk.DISABLED)
        stop_button.config(state=tk.NORMAL)  # Activar el botón "Detener Cámara"

        # Iniciar un hilo para la captura de la cámara
        camera_thread = Thread(target=capturar_camara)
        camera_thread.start()

        # Restablecer la detección de número de celular al iniciar la cámara
        celular_detectado = ""

def detener_camara():
    global camera_active
    camera_active = False

    # Habilitar el botón "Iniciar Cámara" después de detener la cámara
    start_button.config(state=tk.NORMAL)
    stop_button.config(state=tk.DISABLED)  # Desactivar el botón "Detener Cámara"

def capturar_camara():
    global last_data, celular_detectado
    try:
        while capture.isOpened() and camera_active:
            ret, frame = capture.read()

            if cv2.waitKey(1) == ord('s'):
                break
            qrDetector = cv2.QRCodeDetector()
            data, bbox, rectifiedImage = qrDetector.detectAndDecode(frame)

            if len(data) > 0:
                print(f'Dato: {data}')
                last_data = data
                cv2.imshow('webCam', rectifiedImage)
                verificar_numero_de_celular(data)
            else:
                cv2.imshow('webCam', frame)
    except Exception as e:
        print(f"Error: {e}")
    finally:
        capture.release()
        cv2.destroyAllWindows()
        detener_camara()

# Función para mostrar el último dato en pantalla
def mostrar_ultimo_dato():
    if last_data:
        last_data_label.config(text=f"Último Dato: {last_data}")

# Función para verificar si el dato contiene un número de teléfono
def verificar_numero_de_celular(data):
    global celular_detectado
    match = re.search(r'\d{10}', data)
    if match:
        celular_detectado = match.group()
        mostrar_celular_detectado()

# Función para mostrar el número de celular detectado y enviar un mensaje de WhatsApp
def mostrar_celular_detectado():
    global celular_detectado
    if celular_detectado:
        celular_label.config(text=f"Número de Celular: {celular_detectado}")
        enviar_datos_por_whatsapp(celular_detectado)

# Función para enviar un mensaje de WhatsApp con la hora
def enviar_datos_por_whatsapp(numero):
    global last_data
    try:
        hora_actual = time.strftime("%H:%M")
        mensaje = f"{last_data} ({hora_actual})"
        kit.sendwhatmsg(f"+{numero}", mensaje, time.localtime().tm_hour, time.localtime().tm_min + 1)
    except Exception as e:
        print(f"Error al enviar el mensaje de WhatsApp: {e}")

# Crear una ventana de Tkinter
root = tk.Tk()
root.title("Lector de QR")

# Crear un botón para iniciar la cámara
start_button = tk.Button(root, text="Iniciar Cámara", command=iniciar_camara)
start_button.pack()

# Crear un botón para detener la cámara
stop_button = tk.Button(root, text="Detener Cámara", command=detener_camara, state=tk.DISABLED)
stop_button.pack()

# Crear un botón para mostrar el último dato
show_last_data_button = tk.Button(root, text="Mostrar Último Dato", command=mostrar_ultimo_dato)
show_last_data_button.pack()

# Crear una etiqueta para mostrar el último dato
last_data_label = tk.Label(root, text="")
last_data_label.pack()

# Crear una etiqueta para mostrar el número de celular detectado
celular_label = tk.Label(root, text="")
celular_label.pack()

root.mainloop()


Error: OpenCV(4.8.1) D:\a\opencv-python\opencv-python\opencv\modules\objdetect\src\qrcode.cpp:31: error: (-215:Assertion failed) !img.empty() in function 'cv::checkQRInputImage'

Error: OpenCV(4.8.1) D:\a\opencv-python\opencv-python\opencv\modules\objdetect\src\qrcode.cpp:31: error: (-215:Assertion failed) !img.empty() in function 'cv::checkQRInputImage'

Dato: n_de_registro:1,n_de_cuenta:1926510, Nombre:Cruz Rios Omar Arturo, celular:5581416489, Sexo:Masculino
In 33 Seconds WhatsApp will open and after 15 Seconds Message will be Delivered!
